In [ ]:
import os
from pathlib import Path
from IPython.display import Image, display
os.chdir("/Users/oliverdaniels-koch/projects/elk-experiments")
out_dir = Path("output")
device = "mps"

In [ ]:
import torch 
import datasets
from transformers import AutoTokenizer
from transformer_lens import HookedTransformer


# Debug set_use_plit_qkv_input

## Outputs are inconistent 

In [ ]:
model = HookedTransformer.from_pretrained("pythia-70m")
# show inconsistency 
test_str = "Hello, world"
out_normal = model(test_str)

# set up hooks
model.set_use_split_qkv_input(True)
out_split = model(test_str)

torch.allclose(out_normal, out_split, atol=1e-2)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-70m into HookedTransformer


True

In [ ]:
torch.abs(out_normal - out_split).max()

tensor(0.0015, device='mps:0', grad_fn=<MaxBackward1>)

## Outupts are inconsistent across batch sizes

seems to work on imdb, but not my other data?



In [ ]:
# load subset of dataset and tokenize
model_name = "pythia-70m"
dataset = datasets.load_dataset("imdb", split="train[:10]")
tokens = model.tokenizer(dataset["text"], padding=True, truncation=True, return_tensors="pt")["input_ids"]
model = HookedTransformer.from_pretrained(model_name)
model.set_use_split_qkv_input(True)
out_batch = model(tokens[:4])
out_single = model(tokens[:1])
assert torch.allclose(out_batch[0], out_single[0], atol=5e-4)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-70m into HookedTransformer


In [ ]:
# set hook
model.set_use_split_qkv_input(True)
out_batch = model(tokens[:4])
out_single = model(tokens[:1])
assert torch.allclose(out_batch[0], out_single[0], atol=5e-4)

In [ ]:
torch.abs(out_batch[0] - out_single[0]).max()

tensor(0., device='mps:0', grad_fn=<MaxBackward1>)

In [ ]:
# load task
from cupbearer import tasks
task = tasks.tiny_natural_mechanisms("hex", "mps", model_name)

Loaded pretrained model attn-only-1l into HookedTransformer
Moving model to device:  mps


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-70m into HookedTransformer
Moving model to device:  mps


In [ ]:
trusted_tokens = torch.stack([torch.tensor(data["prefix_tokens"]) for data in task.trusted_data.data])
untrusted_clean_tokens = torch.stack([torch.tensor(data["prefix_tokens"]) for data in task.test_data.normal_data.data])
anomalous_tokens = torch.stack([torch.tensor(data["prefix_tokens"]) for data in task.test_data.anomalous_data.data])

In [ ]:
model = HookedTransformer.from_pretrained(model_name)
out_batch = model(trusted_tokens[:4])
out_single = model(trusted_tokens[:1])
assert torch.allclose(out_batch[0], out_single[0], atol=5e-4)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-70m into HookedTransformer


In [ ]:
model_name = "pythia-70m"

In [ ]:
# set hook
input = torch.tensor([
    [2, 2, 2, 69, 26, 67, 17, 14, 14836, 3593, 14, 21, 12347, 14, 1257, 24],
    [535, 50270, 338, 1881, 15, 2364, 15, 25950, 2073, 15741, 64, 29786,  3401, 35495, 686, 26]
])
model_name = "pythia-70m"
model = HookedTransformer.from_pretrained(model_name)
model.set_use_split_qkv_input(True)
out_batch = model(input[:2])
out_single = model(input[:1])
assert torch.allclose(out_batch[0], out_single[0], atol=1e-3), torch.abs(out_batch[0] - out_single[0]).max()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-70m into HookedTransformer


AssertionError: tensor(0.0052, device='mps:0', grad_fn=<MaxBackward1>)

In [ ]:
# add padding token to all tokens 
trusted_tokens_padded = torch.cat([trusted_tokens, torch.zeros_like(trusted_tokens[:, :1])], dim=1)

In [ ]:
model = HookedTransformer.from_pretrained(model_name)
model.set_use_split_qkv_input(True)
out_batch = model(trusted_tokens_padded[:4])
out_single = model(trusted_tokens_padded[:1])
assert torch.allclose(out_batch[0], out_single[0], atol=5e-4), torch.abs(out_batch[0] - out_single[0]).max()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-70m into HookedTransformer


AssertionError: tensor(0.0052, device='mps:0', grad_fn=<MaxBackward1>)